In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用预构建容器（曾经的托管运行时）训练和部署 SKLearn 模型

## 安装

安装Google的*云存储*库。

In [ ]:
! pip3 install google-cloud-storage

重新启动内核

安装了Vertex SDK和Google *cloud-storage*后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

### GPU 运行时

*如果有这个选项的话，请确保在 GPU 运行时下运行这个笔记本。在 Colab 中，选择* **运行时 > 更改运行类型 > GPU**

### 设置您的 GCP 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额度用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已安装在 Google Cloud 笔记本中。

5. 在下方的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 中以 `!` 开头的行将作为 shell 命令运行，并且会将以 `$` 开头的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量在本笔记本的其余部分中使用。以下是Vertex支持的区域。我们建议尽可能选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶来训练Vertex。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex服务的区域支持]()。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加一个直播教程会话，您可能会使用一个共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的GCP账户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。请跳过这一步。

*注意：如果您在Vertex笔记本上运行单元格，则单元格知道要跳过执行身份验证步骤。*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### 创建云存储桶

**无论您使用哪种笔记本环境，下面的步骤都是必需的。**

本教程旨在使用公共云存储桶中的训练数据，并为您的批量预测使用本地云存储桶。您也可以使用您自己存储在本地云存储桶中的训练数据。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问。

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

#### 导入Vertex SDK

将Vertex SDK导入到我们的Python环境中。

In [ ]:
import json
import os
import sys
import time

from googleapiclient import discovery

顶点常量

为顶点设置以下常量：

- `PARENT`：数据集、模型和端点资源的顶点位置根路径。

In [ ]:
# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

客户端

Vertex SDK采用客户端/服务器模型。在您的一侧（即Python脚本），您将创建一个客户端，用于向服务器（Vertex）发送请求并接收响应。

在本教程中，您将使用多个客户端，因此请提前将它们全部设置好。

In [ ]:
client = discovery.build("ml", "v1")

准备一个训练师的脚本

### 包装组装

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
setuptools.setup(\n\
    install_requires=[\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom Census Income\n\
Version: 0.0.0\n\
Summary: Demonstration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 任务.py的内容

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Census Income

from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
import datetime
import pandas as pd

from google.cloud import storage

import numpy as np
import argparse
import os
import sys

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))

# Public bucket holding the census data
bucket = storage.Client().bucket('cloud-samples-data')

# Path to the data inside the public bucket
blob = bucket.blob('ai-platform/sklearn/census_data/adult.data')
# Download the data
blob.download_to_filename('adult.data')

# Define the format of your input data including unused columns (These are the columns from the census data files)
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)

# Categorical columns are columns that need to be turned into a numerical value to be used by scikit-learn
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)


# Load the training census dataset
with open('./adult.data', 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)

# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
train_features = raw_training_data.drop('income-level', axis=1).values.tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
train_labels = (raw_training_data['income-level'] == ' >50K').values.tolist()

# Since the census data set has categorical features, we need to convert
# them to numerical values. We'll use a list of pipelines to convert each
# categorical column and then use FeatureUnion to combine them before calling
# the RandomForestClassifier.
categorical_pipelines = []

# Each categorical column needs to be extracted individually and converted to a numerical value.
# To do this, each categorical column will use a pipeline that extracts one feature column via
# SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
# A scores array (created below) will select and extract the feature column. The scores array is
# created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
for i, col in enumerate(COLUMNS[:-1]):
    if col in CATEGORICAL_COLUMNS:
        # Create a scores array to get the individual categorical column.
        # Example:
        #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical',
        #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
        #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #
        # Returns: [['State-gov']]
        # Build the scores array.
        scores = [0] * len(COLUMNS[:-1])
        # This column is the categorical column we want to extract.
        scores[i] = 1
        skb = SelectKBest(k=1)
        skb.scores_ = scores
        # Convert the categorical column to a numerical value
        lbn = LabelBinarizer()
        r = skb.transform(train_features)
        lbn.fit(r)
        # Create the pipeline to extract the categorical feature
        categorical_pipelines.append(
            ('categorical-{}'.format(i), Pipeline([
                ('SKB-{}'.format(i), skb),
                ('LBN-{}'.format(i), lbn)])))
        
# Create pipeline to extract the numerical features
skb = SelectKBest(k=6)
# From COLUMNS use the features that are numerical
skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
categorical_pipelines.append(('numerical', skb))

# Combine all the features using FeatureUnion
preprocess = FeatureUnion(categorical_pipelines)

# Create the classifier
classifier = RandomForestClassifier()

# Transform the features and fit them to the classifier
classifier.fit(preprocess.transform(train_features), train_labels)

# Create the overall model as a single pipeline
pipeline = Pipeline([
    ('union', preprocess),
    ('classifier', classifier)
])

# Split path into bucket and subdirectory
bucket = args.model_dir.split('/')[2]
subdir = args.model_dir.split('/')[-1]

# Write model to a local file
joblib.dump(pipeline, 'model.joblib')

# Upload the model to GCS
bucket = storage.Client().bucket(bucket)
blob = bucket.blob(subdir + '/model.joblib')
blob.upload_from_filename('model.joblib')


请将商店培训脚本存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/census.tar.gz

训练一个模型

### [项目.作业.创建](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/create)

请求

In [ ]:
JOB_NAME = "custom_job_SKL" + TIMESTAMP

training_input = {
    "scaleTier": "BASIC",
    "packageUris": ["gs://" + BUCKET_NAME + "/census.tar.gz"],
    "pythonModule": "trainer.task",
    "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
    "region": REGION,
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7",
}

body = {"jobId": JOB_NAME, "trainingInput": training_input}

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID)
request.body = body

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID, body=body)

{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/jobs?alt=json",
  "method": "POST",
  "body": {
    "jobId": "custom_job_SKL20210302140139",
    "trainingInput": {
      "scaleTier": "BASIC",
      "packageUris": [
        "gs://migration-ucaip-trainingaip-20210302140139/census.tar.gz"
      ],
      "pythonModule": "trainer.task",
      "args": [
        "--model-dir=gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139"
      ],
      "region": "us-central1",
      "runtimeVersion": "2.4",
      "pythonVersion": "3.7"
    }
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.jobs.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}

#### 电话

In [ ]:
result = request.execute()

回应

In [ ]:
print(json.dumps(result, indent=2))

{
  "jobId": "custom_job_SKL20210302140139",
  "trainingInput": {
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210302140139/census.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139"
    ],
    "region": "us-central1",
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-03-02T14:09:33Z",
  "state": "QUEUED",
  "trainingOutput": {},
  "etag": "YQ/mo0C8EUg="
}

In [ ]:
# The short numeric ID for the custom training job
custom_training_short_id = result["jobId"]
# The full unique ID for the custom training job
custom_training_id = "projects/" + PROJECT_ID + "/jobs/" + result["jobId"]

print(custom_training_id)

获取项目中的作业。

###电话

In [ ]:
request = client.projects().jobs().get(name=custom_training_id)

result = request.execute()

#### 回应

In [ ]:
print(json.dumps(result, indent=2))

{
  "jobId": "custom_job_SKL20210302140139",
  "trainingInput": {
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210302140139/census.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139"
    ],
    "region": "us-central1",
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-03-02T14:09:33Z",
  "state": "PREPARING",
  "trainingOutput": {},
  "etag": "/X2Bt4OWbWU="
}

In [ ]:
while True:
    response = client.projects().jobs().get(name=custom_training_id).execute()

    if response["state"] != "SUCCEEDED":
        print("Training job has not completed:", response["state"])
        if response["state"] == "FAILED":
            break
    else:
        break
    time.sleep(60)

# model artifact output directory on Google Cloud Storage
model_artifact_dir = response["trainingInput"]["args"][0].split("=")[-1]
print("artifact location  " + model_artifact_dir)

部署模型

### [projects.models.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models/create)

### [projects.models.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models/create)

#### 请求

In [ ]:
body = {"name": "custom_job_SKL" + TIMESTAMP}

request = client.projects().models().create(parent="projects/" + PROJECT_ID)
request.body = json.loads(json.dumps(body, indent=2))

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().models().create(parent="projects/" + PROJECT_ID, body=body)

{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models?alt=json",
  "method": "POST",
  "body": {
    "name": "custom_job_SKL20210302140139"
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.models.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}

电话

In [ ]:
result = request.execute()

####回复

In [ ]:
print(json.dumps(result, indent=2))

*示例输出*：
```
{
  "name": "projects/migration-ucaip-training/models/custom_job_SKL20210302140139",
  "regions": [
    "us-central1"
  ],
  "etag": "Lmd8u9MSSIA="
}
```

In [ ]:
model_id = result["name"]

### [projects.models.versions.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/create)

### [projects.models.versions.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/create)

请求

In [ ]:
version = {
    "name": "custom_job_SKL" + TIMESTAMP,
    "deploymentUri": model_artifact_dir,
    "runtimeVersion": "2.1",
    "framework": "SCIKIT_LEARN",
    "pythonVersion": "3.7",
    "machineType": "mls1-c1-m2",
}

request = client.projects().models().versions().create(parent=model_id)
request.body = version

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().models().versions().create(parent=model_id, body=version)

*示例输出*：
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models/custom_job_SKL20210302140139/versions?alt=json",
  "method": "POST",
  "body": {
    "name": "custom_job_SKL20210302140139",
    "deploymentUri": "gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139",
    "runtimeVersion": "2.1",
    "framework": "SCIKIT_LEARN",
    "pythonVersion": "3.7",
    "machineType": "mls1-c1-m2"
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.models.versions.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```

#### 电话

In [ ]:
result = request.execute()

### 回复

In [ ]:
print(json.dumps(result, indent=2))

{
  "name": "projects/migration-ucaip-training/operations/create_custom_job_SKL20210302140139_custom_job_SKL20210302140139-1614695138432",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2021-03-02T14:25:38Z",
    "operationType": "CREATE_VERSION",
    "modelName": "projects/migration-ucaip-training/models/custom_job_SKL20210302140139",
    "version": {
      "name": "projects/migration-ucaip-training/models/custom_job_SKL20210302140139/versions/custom_job_SKL20210302140139",
      "deploymentUri": "gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139",
      "createTime": "2021-03-02T14:25:38Z",
      "runtimeVersion": "2.1",
      "etag": "ilPQVTiR+IM=",
      "framework": "SCIKIT_LEARN",
      "machineType": "mls1-c1-m2",
      "pythonVersion": "3.7"
    }
  }
}

In [ ]:
# The full unique ID for the model version
model_version_name = result["metadata"]["version"]["name"]

print(model_version_name)

In [ ]:
while True:
    response = (
        client.projects().models().versions().get(name=model_version_name).execute()
    )
    if response["state"] == "READY":
        print("Model version created.")
        break
    time.sleep(60)

进行批量预测

批量预测只支持Tensorflow。FRAMEWORK_SCIKIT_LEARN目前不可用。

进行在线预测

### 为在线预测准备数据项

In [ ]:
INSTANCES = [
    [
        25,
        "Private",
        226802,
        "11th",
        7,
        "Never-married",
        "Machine-op-inspct",
        "Own-child",
        "Black",
        "Male",
        0,
        0,
        40,
        "United-States",
    ],
    [
        38,
        "Private",
        89814,
        "HS-grad",
        9,
        "Married-civ-spouse",
        "Farming-fishing",
        "Husband",
        "White",
        "Male",
        0,
        0,
        50,
        "United-States",
    ],
    [
        28,
        "Local-gov",
        336951,
        "Assoc-acdm",
        12,
        "Married-civ-spouse",
        "Protective-serv",
        "Husband",
        "White",
        "Male",
        0,
        0,
        40,
        "United-States",
    ],
    [
        44,
        "Private",
        160323,
        "Some-college",
        10,
        "Married-civ-spouse",
        "Machine-op-inspct",
        "Husband",
        "Black",
        "Male",
        7688,
        0,
        40,
        "United-States",
    ],
    [
        18,
        "?",
        103497,
        "Some-college",
        10,
        "Never-married",
        "?",
        "Own-child",
        "White",
        "Female",
        0,
        0,
        30,
        "United-States",
    ],
    [
        34,
        "Private",
        198693,
        "10th",
        6,
        "Never-married",
        "Other-service",
        "Not-in-family",
        "White",
        "Male",
        0,
        0,
        30,
        "United-States",
    ],
    [
        29,
        "?",
        227026,
        "HS-grad",
        9,
        "Never-married",
        "?",
        "Unmarried",
        "Black",
        "Male",
        0,
        0,
        40,
        "United-States",
    ],
    [
        63,
        "Self-emp-not-inc",
        104626,
        "Prof-school",
        15,
        "Married-civ-spouse",
        "Prof-specialty",
        "Husband",
        "White",
        "Male",
        3103,
        0,
        32,
        "United-States",
    ],
    [
        24,
        "Private",
        369667,
        "Some-college",
        10,
        "Never-married",
        "Other-service",
        "Unmarried",
        "White",
        "Female",
        0,
        0,
        40,
        "United-States",
    ],
    [
        55,
        "Private",
        104996,
        "7th-8th",
        4,
        "Married-civ-spouse",
        "Craft-repair",
        "Husband",
        "White",
        "Male",
        0,
        0,
        10,
        "United-States",
    ],
]

### [projects.predict](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects/predict)

### [projects.predict](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects/predict)

请求

In [ ]:
request = client.projects().predict(name=model_version_name)
request.body = json.loads(json.dumps({"instances": INSTANCES}, indent=2))

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().predict(
    name=model_version_name, body={"instances": INSTANCES}
)

*示例输出*：
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models/custom_job_SKL20210302140139/versions/custom_job_SKL20210302140139:predict?alt=json",
  "method": "POST",
  "body": {
    "instances": [
      [
        25,
        "私人",
        226802,
        "11th",
        7,
        "从未结婚",
        "机器操作检查员",
        "自己的孩子",
        "黑色",
        "男性",
        0,
        0,
        40,
        "美国"
      ],
      [
        38,
        "私人",
        89814,
        "高中毕业",
        9,
        "已婚市民配偶",
        "农业-渔业",
        "丈夫",
        "白种人",
        "男性",
        0,
        0,
        50,
        "美国"
      ],
      [
        28,
        "地方政府",
        336951,
        "大学准学士",
        12,
        "已婚市民配偶",
        "保护服务",
        "丈夫",
        "白种人",
        "男性",
        0,
        0,
        40,
        "美国"
      ],
      [
        44,
        "私人",
        160323,
        "一些大学",
        10,
        "已婚市民配偶",
        "机器操作检查员",
        "丈夫",
        "黑色",
        "男性",
        7688,
        0,
        40,
        "美国"
      ],
      [
        18,
        "?",
        103497,
        "一些大学",
        10,
        "从未结婚",
        "?",
        "自己的孩子",
        "白种人",
        "女性",
        0,
        0,
        30,
        "美国"
      ],
      [
        34,
        "私人",
        198693,
        "10th",
        6,
        "从未结婚",
        "其他服务",
        "不是家人",
        "白种人",
        "男性",
        0,
        0,
        30,
        "美国"
      ],
      [
        29,
        "?",
        227026,
        "高中毕业",
        9,
        "从未结婚",
        "?",
        "未婚",
        "黑色",
        "男性",
        0,
        0,
        40,
        "美国"
      ],
      [
        63,
        "个体经营者非公",
        104626,
        "专业学校",
        15,
        "已婚市民配偶",
        "专业特长",
        "丈夫",
        "白种人",
        "男性",
        3103,
        0,
        32,
        "美国"
      ],
      [
        24,
        "私人",
        369667,
        "一些大学",
        10,
        "从未结婚",
        "其他服务",
        "未婚",
        "白种人",
        "女性",
        0,
        0,
        40,
        "美国"
      ],
      [
        55,
        "私人",
        104996,
        "7-8年级",
        4,
        "已婚市民配偶",
        "技术维修",
        "丈夫",
        "白种人",
        "男性",
        0,
        0,
        10,
        "美国"
      ]
    ]
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.predict",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```

####电话

In [ ]:
result = request.execute()

#### 回复

In [ ]:
print(json.dumps(result, indent=2))

*Chinese Translation*:
```
{
  "predictions": [
    false,
    false,
    false,
    false,
    false,
    false,
    false,
    false,
    false,
    false
  ]
}
```

### [projects.models.versions.delete](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/delete)

### [projects.models.versions.delete](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/delete)

### 请求

In [ ]:
request = client.projects().models().versions().delete(name=model_version_name)

#### 电话

In [ ]:
response = request.execute()

回复

In [ ]:
print(json.dumps(response, indent=2))

{
  "name": "projects/migration-ucaip-training/operations/delete_custom_job_SKL20210302140139_custom_job_SKL20210302140139-1614695211809",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2021-03-02T14:26:51Z",
    "operationType": "DELETE_VERSION",
    "modelName": "projects/migration-ucaip-training/models/custom_job_SKL20210302140139",
    "version": {
      "name": "projects/migration-ucaip-training/models/custom_job_SKL20210302140139/versions/custom_job_SKL20210302140139",
      "deploymentUri": "gs://migration-ucaip-trainingaip-20210302140139/custom_job_SKL20210302140139",
      "createTime": "2021-03-02T14:25:38Z",
      "runtimeVersion": "2.1",
      "state": "READY",
      "etag": "5R4YqeqWMk8=",
      "framework": "SCIKIT_LEARN",
      "machineType": "mls1-c1-m2",
      "pythonVersion": "3.7"
    }
  }
}

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
delete_model = True
delete_bucket = True

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model:
        client.projects().models().delete(name=model_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME